In [115]:
import csv
import cv2
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from copy import deepcopy
import tensorflow as tf
import tensorflow.keras.layers as tfl
# from __future__ import print_function, division
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
import pandas as pd

In [116]:
TRAIN_PATH = "./train.csv"#/kaggle/input/captcha-hacker/train
TEST_PATH = "./test.csv"#/kaggle/input/captcha-hacker/test
NUM_EPOCHS1_and_2 = 500
NUM_EPOCHS3 = 500
BATCH_SIZE = 16
device = torch.device("cuda:0")



In [117]:
class TestDataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = data

        # print(self.data)

        # self.data = make_dataset(TRAIN_PATH, alphabet, 36, 2)
        self.return_filename = return_filename
        self.root = root

    def __getitem__(self, index):
        data = self.data[index]
        return torch.Tensor(data)

    def __len__(self):
        return len(self.data)

In [118]:
def clean(data):
    le = LabelEncoder()

    # Replace str by numbers
    data = deepcopy(data)
    cols = ["attribute_0"] + ["attribute_1"]
    for col in cols:
        data[col] = le.fit_transform(data[col])
        #print(le.classes_)

    data = data.drop(['id', 'product_code'], axis=1)

    # None values
    imputer = SimpleImputer(strategy='mean')
    final_data = pd.DataFrame(imputer.fit_transform(data))

    final_data.columns = data.columns

    return final_data

In [119]:
test_data = pd.read_csv(TEST_PATH)

test_not_clean = test_data
test_clean = clean(test_data)
test = test_clean.to_numpy()

test_ds = TestDataset(test, root=TEST_PATH)
test_dl = DataLoader(test_ds, batch_size=1, drop_last=False, shuffle=False)

In [120]:
class Model(nn.Module):
    def __init__(self, input_shape):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_shape, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [121]:
model = torch.load('./model1.pt')
model.eval()
pred = []
for x in tqdm(test_dl):
    # print(x)
    y_pred = model(x)
    output = torch.sigmoid(y_pred)
    output = output.cpu().detach().numpy()
    for i in range(len(output)):
        pred.append(output[i][0])
result = pd.DataFrame({'id': test_not_clean['id'], 'failure': pred})
result.to_csv('submission.csv', index=0)
result


  0%|          | 0/20775 [00:00<?, ?it/s]

,id,failure
0,26570,0.555318
1,26571,0.546293
2,26572,0.551362
3,26573,0.554308
4,26574,0.589645
...,...,...
20770,47340,0.562295
20771,47341,0.541249
20772,47342,0.537592
20773,47343,0.557818
